In [7]:
%%bash
# Clone the GitHub repo
git clone https://github.com/isacancabuk/toyota-model-labelling.git /content/toyota-model-labelling
cd /content/toyota-model-labelling

# Install Python packages from your cleaned requirements.txt
pip install -q -r requirements.txt

fatal: destination path '/content/toyota-model-labelling' already exists and is not an empty directory.


In [8]:
# Cell 2: one-time imports and dataset inspection
import os
from pathlib import Path

REPO = Path('/content/toyota-model-labelling')
DATA_4R = REPO / 'data' / '4runner'

if not DATA_4R.exists():
    raise FileNotFoundError(f"Folder not found: {DATA_4R}")

files = list(DATA_4R.glob('*.jpg'))
print("Total 4Runner images:", len(files))
print("First 5 filenames:", [f.name for f in files[:5]])

Total 4Runner images: 946
First 5 filenames: ['4runner-1586451608.jpg', '4runner-1614916643.jpg', '4runner-468347014.jpg', '4runner-1768807010.jpg', '4runner-959180871.jpg']


In [9]:
# Cell 3: split the 4Runner images into train/val (80/20)
import random, shutil

SRC = REPO / 'data' / '4runner'
TRAIN = REPO / 'data' / '4runner_split' / 'train'
VAL   = REPO / 'data' / '4runner_split' / 'val'

for d in (TRAIN, VAL):
    d.mkdir(parents=True, exist_ok=True)

imgs = list(SRC.glob('*.jpg'))
random.shuffle(imgs)
cut = int(0.8 * len(imgs))

for i, img in enumerate(imgs):
    tgt = TRAIN if i < cut else VAL
    shutil.copy(img, tgt / img.name)

print(f"Train count: {len(list(TRAIN.glob('*')))}")
print(f" Val count: {len(list(VAL.glob('*')))}")

Train count: 756
 Val count: 190


In [10]:
%%bash
# Cell 4: get YOLOv5 code & install its requirements
git clone https://github.com/ultralytics/yolov5.git /content/yolov5
cd /content/yolov5
pip install -q -r requirements.txt

Cloning into '/content/yolov5'...


In [16]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Cell 5: baseline recall for 'car' on your Val set (fixed conf handling)
import torch
from pathlib import Path

# 1) Load the COCO-pretrained YOLOv5s
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# 2) Set minimum confidence once
model.conf = 0.25  # equivalent to passing conf_thres=0.25 each call

CAR_ID = 2
val_images = list((REPO / 'data' / '4runner_split' / 'val').glob('*.jpg'))
total = len(val_images)
detected = 0

# 3) Loop and count
for img in val_images:
    results = model(str(img))         # now uses model.conf threshold
    df = results.pandas().xyxy[0]     # DataFrame of all detections
    if (df['class'] == CAR_ID).any():
        detected += 1

# 4) Report recall
print(f"Detected in {detected}/{total} images → Recall: {detected/total:.2%}")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-5-12 Python-3.11.12 torch-2.6.0+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Detected in 101/190 images → Recall: 53.16%


In [20]:
%%bash
# Search every folder under /content for 4runner.yaml
echo "Searching for 4runner.yaml..."
find /content -type f -name 4runner.yaml

Searching for 4runner.yaml...
/content/toyota-model-labelling/data/4runner.yaml


In [22]:
%%bash
# Make sure the target data/ folder exists
mkdir -p /content/toyota-model-labelling/data

# Copy the YAML into the place YOLO expects
cp "/content/toyota-model-labelling/data/4runner.yaml" /content/toyota-model-labelling/data/4runner.yaml

# Verify it’s now in the right spot
ls -l /content/toyota-model-labelling/data/4runner.yaml

-rw-r--r-- 1 root root 437 May 12 21:00 /content/toyota-model-labelling/data/4runner.yaml


cp: '/content/toyota-model-labelling/data/4runner.yaml' and '/content/toyota-model-labelling/data/4runner.yaml' are the same file


In [24]:
%%bash
cat > /content/toyota-model-labelling/data/4runner_abs.yaml << 'EOF'
train: /content/toyota-model-labelling/data/4runner_split/train
val:   /content/toyota-model-labelling/data/4runner_split/val

nc: 1
names: ['4runner']
EOF

In [25]:
%%bash
export WANDB_MODE=disabled
cd /content/yolov5

python train.py \
  --img 640 \
  --batch 8 \
  --epochs 50 \
  --data /content/toyota-model-labelling/data/4runner_abs.yaml \
  --weights yolov5s.pt \
  --name 4runner_finetune \
  --cache

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-05-12 21:37:07.339511: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747085827.374276   23617 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747085827.385084   23617 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=/content/toyota-model-labelling/data/4runner_abs.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=8, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_pop

CalledProcessError: Command 'b'export WANDB_MODE=disabled\ncd /content/yolov5\n\npython train.py \\\n  --img 640 \\\n  --batch 8 \\\n  --epochs 50 \\\n  --data /content/toyota-model-labelling/data/4runner_abs.yaml \\\n  --weights yolov5s.pt \\\n  --name 4runner_finetune \\\n  --cache\n'' returned non-zero exit status 1.